In [8]:
import os
from langchain_openai import ChatOpenAI
from langchain.agents import load_tools
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
from dotenv import load_dotenv
from langchain import hub
from langchain_community.tools.google_finance import GoogleFinanceQueryRun
from langchain_community.utilities.google_finance import GoogleFinanceAPIWrapper
from langchain_community.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain_openai import AzureChatOpenAI


 
load_dotenv()

os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("SIX_OPENAI_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("SIX_OPENAI_URL")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERP_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

In [9]:
tool = GoogleFinanceQueryRun(api_wrapper=GoogleFinanceAPIWrapper())

params = {
    "engine": "bing",
    "gl": "us",
    "hl": "en",
}
search = SerpAPIWrapper(params=params)


prompt = hub.pull("hwchase17/react")


#llm = ChatOpenAI(model_name="gpt-3.5-turbo" ,temperature=0)
llm = AzureChatOpenAI(
    openai_api_version=os.getenv("SIX_OPENAI_VERSION"),
    azure_deployment=os.getenv("SIX_OPENAI_MODEL"),
)


tools = load_tools([
#    "google-finance",
      "google-serper"
        #"serpapi"
        ], llm=llm, allow_dangerous_tools=True)
agent = create_react_agent(llm, tools, prompt)


In [12]:
prompt_template = PromptTemplate(
    template=""""You are a financial data assistent for retrieving data about funds from the web./ 
Please specifically answer what company is associated with an ISIN, retrieve their name, fund makeup, environmental, social and governance scores as well as their current price.
Retrieve data for the following ISIN: {isin}./
Scrape the data from 
Return what you could retrieve in a JSON format with no additional text.""",
    input_variables=["isin"]
)

In [11]:
isin = "CH0281016474"
full_prompt = prompt_template.format(isin=isin)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": full_prompt})




> Entering new AgentExecutor chain...
To answer this question, I need to gather financial data about the fund associated with the ISIN CH0281016474. This will likely involve finding the company name, the components of the fund (fund makeup), the ESG (environmental, social, governance) scores, and the current price of the fund. Since this information is typically available from financial databases or investment platforms, I will use the google_serper tool to find the relevant information online.

Action: google_serper
Action Input: CH0281016474 fund informationCH0281016474. Valor: 28101647. 96.93. NAV. As at 14 Mar 2024. 1.32%. Performance YTD. Overview; Performance; Key Facts; Breakdowns; Documents; Risks & ... What is this product? Type. The fund is a contractual umbrella fund under Swiss law of the type “Other funds for traditional investments” pursuant to the ... CH0281016474 / 28101647. Ongoing charges ... fund/fund units or any ... All data contained herein, including fund infor

{'input': '"You are a financial data assistent for retrieving data about funds from the web./ \nPlease specifically answer what company is associated with an ISIN, retrieve their name, fund makeup, environmental, social and governance scores as well as their current price.\nRetrieve data for the following ISIN: CH0281016474./\nReturn what you could retrieve in a JSON format with no additional text.',
 'output': '```json\n{\n  "ISIN": "CH0281016474",\n  "Company": "Bank Vontobel AG",\n  "FundName": "Vontobel Fund (CH) - Pension Invest Yield A",\n  "FundMakeup": "Information not found",\n  "ESGScores": "Information not found",\n  "CurrentPrice": "Information not found"\n}\n```'}